## Q1. What is the dlt version?

In [1]:
pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

Note: you may need to restart the kernel to use updated packages.


In [8]:
import dlt
print(dlt.__version__)

1.12.3


In [1]:
@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

In [4]:
zoomcamp_data()

<generator object zoomcamp_data at 0x76db743fdf40>

##  Q2. How many rows were inserted into the zoomcamp_data collection?

In [11]:
from dlt.destinations import qdrant
import requests

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [12]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Run started at 2025-07-08 12:29:00.905118+00:00 and COMPLETED in 9.02 seconds with 4 steps.
Step extract COMPLETED in 1.04 seconds.

Load package 1751977741.922287 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.08 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- zoomcamp_data: 948 row(s)

Load package 1751977741.922287 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 6.88 seconds.
Pipeline zoomcamp_pipeline load step completed in 6.87 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /workspaces/llm-zoomcamp-2025/dlt-workshop/db.qdrant location to store data
Load package 1751977741.922287 is LOADED and contains no failed jobs

Step run COMPLETED in 9.02 seconds.
Pipeline zoomcamp_pipeline load step completed in 6.87 seconds
1 load package(s) were loaded to des

In [25]:
import pandas as pd
df = pd.read_json('db.qdrant/meta.json')
df

,collections,aliases
zoomcamp_tagged_data,{'vectors': {'fast-bge-small-en': {'size': 384...,NaN
zoomcamp_tagged_data__dlt_loads,{'vectors': {'fast-bge-small-en': {'size': 384...,NaN
zoomcamp_tagged_data__dlt_pipeline_state,{'vectors': {'fast-bge-small-en': {'size': 384...,NaN
zoomcamp_tagged_data_zoomcamp_data,{'vectors': {'fast-bge-small-en': {'size': 384...,NaN
zoomcamp_tagged_data__dlt_version,{'vectors': {'fast-bge-small-en': {'size': 384...,NaN
